# Kuan Chen Chen Tic_Tac_Toe_with_ml
## Made by: 2024/10/27 07:08PM
## Project instruction:
### Playing Tic-Tac-Toe by enter the avalible input.If enter invalid input the program will tell you to enter the correct one,the program will also detect the win or tie, and the user can choose to end the game or repeat the game again and again. Beside this there is a AI that can play the tic tac toe with you

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import random

In [3]:
class Board:
    def __init__(self):
        self.board = [
            ["R\\C", " 0 ", " 1 ", " 2 "],
            [" 0 ", "   ", "   ", "   "],
            [" 1 ", "   ", "   ", "   "],
            [" 2 ", "   ", "   ", "   "]
        ]

    def printBoard(self):
        for row in self.board:
            print(" | ".join(row)+" | ")
            print("-----------------------")

    def updateBoard(self, row, col, symbol):
        self.board[row][col] = f" {symbol} "

    def getCell(self, row, col):
        return self.board[row][col]

    def convert_to_ml_state(self):
        # Convert board state to ML format
        state = []
        for i in range(1, 4):
            for j in range(1, 4):
                cell = self.getCell(i, j).strip()
                if cell == 'X':
                    state.append(1)
                elif cell == 'O':
                    state.append(-1)
                else:
                    state.append(0)
        return state

In [11]:
class MLGame:
    def __init__(self):
        self.board = Board()
        self.turn = 'X'
        self.model = self.train_model()

    def train_model(self):
        try:
            # Load training data
            data = []
            with open('tictac_final.txt', 'r') as file:
                for line in file:
                    values = [int(x) for x in line.strip().split()]
                    data.append(values)

            data = np.array(data)
            X = data[:, :9]  # First 9 columns are features (board states)
            y = data[:, 9]   # Last column is label (result)

            # Train Random Forest model
            model = RandomForestClassifier(n_estimators=100, random_state=42)
            model.fit(X, y)
            print("ML model training successful!")
            return model
        except Exception as e:
            print(f"ML model training failed: {e}")
            return None

    def switchPlayer(self):
        self.turn = 'O' if self.turn == 'X' else 'X'

    def validateEntry(self, row, col):
        if 1 <= row <= 3 and 1 <= col <= 3:
            if self.board.getCell(row, col) != "   ":
                print("That cell is already taken.\nPlease make another selection.")
                return False
            return True
        print("Invalid entry: try again.\nRow & column numbers must be either 0, 1, or 2.")
        return False

    def checkFull(self):
        for i in range(1, 4):
            for j in range(1, 4):
                if self.board.getCell(i, j) == "   ":
                    return False
        return True

    def checkWin(self):
        # Check rows
        for i in range(1, 4):
            if (self.board.getCell(i, 1) ==
                self.board.getCell(i, 2) ==
                self.board.getCell(i, 3) == f" {self.turn} "):
                return True

        # Check columns
        for j in range(1, 4):
            if (self.board.getCell(1, j) ==
                self.board.getCell(2, j) ==
                self.board.getCell(3, j) == f" {self.turn} "):
                return True

        # Check diagonals
        if (self.board.getCell(1, 1) ==
            self.board.getCell(2, 2) ==
            self.board.getCell(3, 3) == f" {self.turn} "):
            return True

        if (self.board.getCell(1, 3) ==
            self.board.getCell(2, 2) ==
            self.board.getCell(3, 1) == f" {self.turn} "):
            return True

        return False

    def checkEnd(self):
        if self.checkWin():
            print(f"{self.turn} IS THE WINNER!!!")
            self.board.printBoard()
            return True

        if self.checkFull():
            print("DRAW! NOBODY WINS!!")
            self.board.printBoard()
            return True

        return False

    def get_ml_move(self):
        """Get move using ML model for every turn"""
        board_state = self.board.convert_to_ml_state()

        if self.model is not None:
            try:
                best_score = float('-inf')
                best_move = None

                # Try each possible move and get prediction
                for i in range(9):
                    if board_state[i] == 0:  # Empty position
                        test_state = board_state.copy()
                        test_state[i] = -1  # AI is O

                        # Get prediction probability
                        prediction = self.model.predict_proba([test_state])[0]

                        # Calculate score based on prediction and position value
                        position_values = [
                            3, 2, 3,  # Corner and side values
                            2, 4, 2,  # Center has highest value
                            3, 2, 3
                        ]

                        # Combine ML prediction with position value
                        score = prediction[1] * position_values[i]

                        # Update best move if current score is higher
                        if score > best_score:
                            best_score = score
                            best_move = ((i // 3) + 1, (i % 3) + 1)

                if best_move:
                    return best_move

            except Exception as e:
                print(f"ML prediction failed: {e}")

        # Fallback strategy if ML fails
        empty_positions = []
        position_values = {
            (2,2): 4,  # Center
            (1,1): 3, (1,3): 3, (3,1): 3, (3,3): 3,  # Corners
            (1,2): 2, (2,1): 2, (2,3): 2, (3,2): 2   # Sides
        }

        for i in range(1, 4):
            for j in range(1, 4):
                if self.board.getCell(i, j) == "   ":
                    empty_positions.append((i, j))

        # Sort positions by their strategic value
        empty_positions.sort(key=lambda x: position_values.get(x, 0), reverse=True)
        return empty_positions[0]

    def playGame(self):
        print("New Game: Human (X) vs AI (O)\n")
        print("-"*23)
        self.board.printBoard()

        while True:
            print(f"\n{self.turn}'s turn")

            if self.turn == 'X':  # Human turn
                print(f"Where do you want your {self.turn} placed?")
                try:
                    user_input = input("Please enter row number and column number separated by a comma.\n").split(",")
                    user_input = [item.strip() for item in user_input if item.strip()]

                    if len(user_input) != 2:
                        print("Invalid input. Please enter exactly two numbers separated by a comma.")
                        continue

                    row, col = map(int, user_input)
                    row += 1
                    col += 1

                    if not self.validateEntry(row, col):
                        continue

                    print(f"You have entered row #{row-1} \n          and column #{col-1}")
                    print("Thank you for your selection.")

                except ValueError:
                    print("Invalid input. Please enter numbers only.")
                    continue
            else:  # AI turn
                print("AI is thinking...")
                row, col = self.get_ml_move()
                print(f"AI plays: {row-1},{col-1}")

            self.board.updateBoard(row, col, self.turn)

            if self.checkEnd():
                break

            print("-"*23)
            self.board.printBoard()
            self.switchPlayer()

In [12]:
def main():
    print("Welcome to ML Tic-tac-toe!")
    repeat = 'yes'
    while repeat[0].lower() == 'y':
        game = MLGame()
        game.playGame()
        repeat = input("\nAnother game? Enter Y or y for yes.\n")
    print("Thanks for playing!")


In [13]:

if __name__ == "__main__":
    main()

Welcome to ML Tic-tac-toe!
ML model training successful!
New Game: Human (X) vs AI (O)

-----------------------
R\C |  0  |  1  |  2  | 
-----------------------
 0  |     |     |     | 
-----------------------
 1  |     |     |     | 
-----------------------
 2  |     |     |     | 
-----------------------

X's turn
Where do you want your X placed?
Please enter row number and column number separated by a comma.
11
Invalid input. Please enter exactly two numbers separated by a comma.

X's turn
Where do you want your X placed?
Please enter row number and column number separated by a comma.
11
Invalid input. Please enter exactly two numbers separated by a comma.

X's turn
Where do you want your X placed?
Please enter row number and column number separated by a comma.
1,1
You have entered row #1 
          and column #1
Thank you for your selection.
-----------------------
R\C |  0  |  1  |  2  | 
-----------------------
 0  |     |     |     | 
-----------------------
 1  |     |  X  |   